In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-GtuuNGDVQjiihrlnHIS_jUqmZY5GysmglbE5_xX6LCPXoJW4u1U2oeJlZLHAeyxmrhUaT3BlbkFJlxN30cH6ZXLqCRlk8oEafWfE-9iNBvHI_WXFFVCsjhEl0sZwcB9VNpzMh9RKkghBJTi0A"

In [2]:
!pip install -q langchain-openai langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 13.1 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [42]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/1e080acd478ebad450075413/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [43]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [44]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1766534402,
 'time_last_update_utc': 'Wed, 24 Dec 2025 00:00:02 +0000',
 'time_next_update_unix': 1766620802,
 'time_next_update_utc': 'Thu, 25 Dec 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 89.6676}

In [45]:
convert.invoke({'base_currency_value':10, 'conversion_rate':89.66})

896.5999999999999

In [46]:
# tool binding
llm = ChatOpenAI()

In [47]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [48]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr')]

In [49]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [50]:
ai_message = llm_with_tools.invoke(messages)

In [51]:
messages.append(ai_message)

In [52]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_ZsOvPcDsbelWa436x8279X38',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_FmHo5zLvBfIw3OYBOu25SDlM',
  'type': 'tool_call'}]

In [53]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [54]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 123, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqChYOi0kZ2tGDbLW2E0wtCZALtk0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4f0e-98a2-7a13-84af-1202c18ca798-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_ZsOvPcDsbelWa436x8279X38', 'type': 'tool_call'}, {'name': 'convert', 'args

In [55]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 89.6676. \n\nBased on this conversion factor, 10 USD is equivalent to approximately 896.68 INR.'